In [75]:
import numpy as np
import pandas as pd
import random as rm

class Die():

    def __init__(self, faces):
        """"" This is the Create_Die docstring"""
        
    # The Initializer Method: Takes an array of faces as an argument.
           
        flag = len(set(faces)) == len(faces)
        if flag == 0:
            raise ValueError("Each value must be Unique.") 
    
        #Faces
        #Asigns FACES to self, the length of the User Inputted array
        self.faces = faces
        self.n_faces = len(faces)
        self.weights = np.ones(len(faces))
    
        #Create Dataframe to save values        
        self.my_die= pd.DataFrame({
            'Face':range(1,self.n_faces+1),
            'Weight':self.weights})


    def change_weight(self, f_change, weight):
        # Changes_Weight Method: changes the weight of a single face provided (x) - Seems to work overall. 
        
        weight = float(weight)
        
        if f_change in self.faces:
            f_change = f_change - 1
            weight = float(weight)
            
            self.weights[f_change]=weight
            self.my_die = pd.DataFrame({
                'Face':range(1,self.n_faces+1),
                'Weight':self.weights})
            print(self.my_die)
        else:
            print("Value is not on the Die.  (i.e. User provided value is not in the array.)")

    def roll_die(self, n_rolls=1):
        Rolled_faces_and_weights_df = self.my_die.sample(n=n_rolls, replace = True, weights = "Weight").reset_index(drop= True)
        return list(Rolled_faces_and_weights_df['Face'])    

                                                
    def show(self, wide_or_narrow = "wide"):
       # A method to show the user the die’s current set of faces and weights (since the latter can be changed). (X)
    
        if wide_or_narrow == 'wide':
            return self.my_die
        
        elif wide_or_narrow =='narrow':
            my_die_narrow = self.my_die.stack().to_frame('Faces')
            my_die_narrow.index.rename(["Roll Number","Dice"], inplace = True)
            return my_die_narrow
        else:
            raise ValueError("Arrgument must be either \"narrow\" or \"wide\"")
    
        print(self.my_die)


In [76]:
class Game(Die): 
    
    def __init__(self,die_list):
        """Game class docstring: A game consists of rolling of one or more dice of the same kind one or more times.
        """      
        self.die_list=die_list       
        self.results = pd.DataFrame()
                        

    def play(self, num_rolls):
        self.my_die = pd.DataFrame()
        self.my_die.index.rename("Roll Number", inplace = True)
        
        for die in range(0, len(self.die_list)):
            self.my_die[die] = self.die_list[die].roll_die(num_rolls)
        
    
    def show(self, wide_or_narrow = "wide"):
       # A method to show the user the die’s current set of faces and weights (since the latter can be changed). (X)
       # Returns the dataframe created in the initializer but possibly updated by the weight changing method. (X)
    
        if wide_or_narrow == 'wide':
            return self.my_die
        
        elif wide_or_narrow =='narrow':
            my_die_narrow = self.my_die.stack().to_frame('Faces')
            my_die_narrow.index.rename(["Roll Number","Dice"], inplace = True)
            return my_die_narrow
        else:
            raise ValueError("Arrgument must be either \"narrow\" or \"wide\"")
    

In [77]:
sixsided = [1,2,3,4,5,6]

testdie1 = Die(sixsided)
testdie2 = Die(sixsided)
testdie3 = Die(sixsided)
testdie4 = Die(sixsided)
testdie5 = Die(sixsided)
testdie6 = Die(sixsided)

testdielist = [testdie1,testdie2,testdie3,testdie4]

testgame = Game(testdielist)

In [83]:
testgame.play(10)

In [82]:
testgame.show("wide")

,0,1,2,3
Roll Number,,,,
0,6,2,3,4
1,1,6,5,1
2,5,6,2,1
3,6,1,6,2
4,4,5,1,3
5,5,5,5,2
6,4,4,2,2
7,3,1,5,4
8,5,1,6,1


In [89]:
class Analyzer(Game): 

    face_count = pd.DataFrame()
    jackpot = pd.DataFrame()
    combos = pd.DataFrame()

    number_of_dice = 0

    game_results = pd.DataFrame()
    temp_df = pd.DataFrame()
    
    def __init__(self, game):
        self.game_results = game.show()
        self.number_of_dice = len(list(self.game_results))
        
    def get_face_count(self):
        self.temp_df = pd.DataFrame(self.game_results.transpose().melt(var_name='rolls',value_name='faces'))
        self.face_counts = pd.crosstab(index=self.temp_df['rolls'],columns=self.temp_df['faces'])
                
    def get_jackpot(self):
        temp_df_2 = self.temp_df.groupby(['rolls','faces'], as_index = False).value_counts()
        self.jackpots = temp_df_2.loc[temp_df_2['count'] == self.number_of_dice]
        return len(self.jackpots)    
    
    def get_combo(self):
        self.combos = self.face_counts.groupby(list(self.face_counts), as_index = False).value_counts()
        
        

In [88]:
#### Testing for Game class ####

first = Die(["1","2","3",4,5,6,7,8,9,10,11,12])
second = Die([1,2,3,4,5,6,7,8,9,10,11,12])
third = Die([1,2,3,4,5,6,7,8,9,10,11,12])

testGame=Game([first,second,third])

testGame.play(4)


testGame.show("wide")


#print(testGame.GameResults)

#dir(testGame)

#type(testGame)
#print(GameResults)

#help(testGame)
#testGame.__dict__

#test=Game([first,second,third])

,0,1,2
Roll Number,,,
0,3,1,7
1,7,12,12
2,9,5,8
3,6,11,3


In [93]:
##Scenario 1
fair_coin = Die(["H","T"])

unfair_coin = Die(["H","T"])
unfair_coin.change_weight("H",5)

#Get this to support alpha-characters.

Number_of_flips = 1000

Fair_coin_set = [fair_coin, fair_coin, fair_coin]
Fair_game = Game(fair_coin_set)
Fair_game.play(number_of_flips)

Fair_game_analyzer = Analyzer(fair_game)
Fair_game_analyzer.get_face_counts()
Fair_game_analyzer.get_combos()

Fair_relative_frequency = fair_game_analyzer.get_jackpots()/number_of_flips

Value_df = pd.DataFrame({"fair":[fair_relative_frequency], "unfair": [unfair_relative_frequency]})

#Graph
Value_df.plot.bar()
Plt.xlabel("Fairness")
Plt.ylabel("Relative Frequency")
Plt.title("Scenario 1: 2-headed coin")


TypeError: unsupported operand type(s) for -: 'str' and 'int'